# BigTransfer (BiT) Fine-Tuning

- 参考資料  
  https://blog.tensorflow.org/2020/05/bigtransfer-bit-state-of-art-transfer-learning-computer-vision.html  
  https://github.com/google-research/big_transfer/tree/master/colabs

- BiTモデルについてる **S, M, L** の意味  
  学習させたデータセットの違い。**L** は非公開。

|  モデル名  |  データセット                 |
|  :--:     |  :---:                      |
|  BiT-S    |  ILSVRC-2012 (1.3M images)  |
|  BiT-M    |  ImageNet-21k (14M images)  |
|  BiT-L    |  JFT (300M images)          |

- BiTモデルについてる **R-??x?** の意味  
  BiTモデルはResNetを利用しているので、そこの情報。R50x3 → 50層のResNetで、各層の幅が通常の3倍。

|  ResNet  |  パラメータ数（概数）  |
|  :----:  |  :----:  |
|  R50x1   |   23M    |
|  R101x1  |   42M    |
|  R50x3   |  211M    |
|  R101x3  |  381M    |
|  R152x4  |  928M    |

パラメータ数確認コード。

```python
model = tfhub.KerasLayer('https://tfhub.dev/google/bit/s-r50x1/1')
print(sum(tf.math.reduce_prod(w.shape).numpy() for w in model.weights))
```

## BiT-HyperRule

ファインチューニングのためのヒューリスティックな方法

### データ拡張・調整

![BigTransfer (BiT): State-of-the-art transfer learning for computer vision; Table1](https://4.bp.blogspot.com/-54vXxLE1bqU/XsMCth3HvzI/AAAAAAAADEs/UVrCWT0o6wYPtCpat81ApNGPus16-CHzgCLcBGAsYHQ/s1600/table1.jpg)
![BigTransfer (BiT): State-of-the-art transfer learning for computer vision; Table2](https://3.bp.blogspot.com/-2JVDJV2A5Uo/XsMT-JfSJtI/AAAAAAAADFQ/UqcoPn11wFAudJJxkdzYxb3tAxBpgoGMQCLcBGAsYHQ/s1600/table%2B2.jpg)

併せて、ランダムに左右反転も入れる。バリデーション用データにはリサイズだけ行えば良い。

- 正解ラベルと乖離が出るので、タスクによっては行わないデータ拡張。
  - 物体の数え上げ ⇒ ランダムクロップはNG
  - 物体の位置特定 ⇒ ランダムフリップはNG


#### MixUp

参考: https://github.com/google-research/big_transfer/blob/master/input_pipeline_tf2_or_jax.py#L118

- データセットに適応するタイミングは **ミニバッチ化`batch()`の後**。
- MixUpが絡むので、ラベルはOne-Hotベクトルにする。
- mixup.ipynbで結果参照。

### バッチサイズ = 512

搭載メモリに合わせて調整する。

### 最適化アルゴリズム = SGD

- Learning rate: 0.003
- Momentum: 0.9

学習率は初期値。学習中の学習率の変更を、以下のスケジューリングを行う。

#### 学習率のスケジューリング

学習の進捗が、全体の30%, 60%, 90%になるタイミングで、学習率を
$ \frac{1}{10} $
ずつ減衰させる。

公式サンプルコードでは、厳密に30%, 60%, 90%で区切っていない。

In [1]:
# -*- coding: utf-8 -*-
import dataclasses
import logging
import pathlib
import json
import itertools
import random

import numpy as np
import tensorflow as tf
import tensorflow_hub as tfhub
import tensorflow_datasets as tfds
import tensorflow_probability as tfp


class WrappedBiT(tf.keras.Model):

    def __init__(self, classes=2, bit_type='m-r50x1', version='1'):
        super().__init__()
        self._head = tf.keras.layers.Dense(classes, kernel_initializer='zeros')
        self._bit = tfhub.KerasLayer(f'https://tfhub.dev/google/bit/{bit_type}/{version}')

    def call(self, inputs, training=None, mask=None):
        features = self._bit(inputs)
        return self._head(features)


@dataclasses.dataclass
class HyperRule:
    schedule_len: int
    resize_edge: int
    crop_size: int
    optimizer: tf.keras.optimizers.Optimizer

    def __init__(self, image_edge: int, dataset_size: int, batch_size=512):
        assert 512 % batch_size == 0
        self.batch_size = batch_size

        if image_edge < 96:
            self.resize_edge, self.crop_size = 160, 128
        else:
            self.resize_edge, self.crop_size = 512, 480

        if dataset_size < 20 * 10 ** 3:
            schedule_len, boundaries = 500, [200, 300, 400]
            self.mixup = lambda image, label: (image, label)  # dummy
        elif 20 * 10 ** 3 <= dataset_size < 500 * 10 ** 3:
            schedule_len, boundaries = 10000, [3000, 6000, 9000]
        else:
            schedule_len, boundaries = 20000, [6000, 12000, 18000]
        self.schedule_len = schedule_len * 512 // batch_size

        lr = 0.003 * batch_size / 512
        lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
            boundaries=boundaries, values=[lr, lr * 1e-1, lr * 1e-2, lr * 1e-3])
        self.optimizer = tf.keras.optimizers.SGD(
            learning_rate=lr_schedule, momentum=0.9)

    def mixup(self, image, label):
        beta_dist = tfp.distributions.Beta(0.1, 0.1)  # alpha = 0.1
        beta = tf.cast(beta_dist.sample([]), tf.float32)
        image = (beta * image + (1 - beta) * tf.reverse(image, axis=[0]))
        label = (beta * label + (1 - beta) * tf.reverse(label, axis=[0]))
        return image, label

### データセットに設定を盛り込む

`setup_dataset_and_hyperrule`

- キャッシュ、シャッフル などHyperRule以外
- HyperRule
  - リサイズ
  - （学習用データのみ）クロップ、左右反転、MixUp

MixUpはバッチ後。挙動などは、mixup.ipynb確認。

In [2]:
def setup_dataset_and_hyperrule(ds_factory, cache_dir, buffer_size=None, batch_size=512):
    def resize_normalize(image, label):
        image = tf.image.resize(image, [rule.resize_edge, rule.resize_edge],
                                method=tf.image.ResizeMethod.BILINEAR)
        image = tf.cast(image, tf.float32) / 255.0
        label = tf.one_hot(label, len(label_names))
        return image, label

    def random_sampling(image, label):
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_crop(image, [rule.crop_size, rule.crop_size, 3])
        return image, label

    (ds_train, ds_train_size), (ds_test, _), label_names, image_edge = ds_factory()

    rule = HyperRule(image_edge=image_edge, dataset_size=ds_train_size, batch_size=batch_size)
    ds_train = (ds_train
                .map(resize_normalize, tf.data.experimental.AUTOTUNE)
                .cache(str(cache_dir / 'train'))
                .shuffle(buffer_size or ds_train_size)
                .repeat()
                .map(random_sampling, tf.data.experimental.AUTOTUNE)
                .batch(rule.batch_size)
                .map(rule.mixup, tf.data.experimental.AUTOTUNE)
                .prefetch(tf.data.experimental.AUTOTUNE))
    ds_test = (ds_test
               .map(resize_normalize, tf.data.experimental.AUTOTUNE)
               .cache(str(cache_dir / 'test'))
               .batch(rule.batch_size)
               .prefetch(tf.data.experimental.AUTOTUNE))

    return rule, ds_train, ds_test, label_names

`dataset_cats_vs_dogs`

カタログ（ https://www.tensorflow.org/datasets/catalog/overview ）から、犬猫を選択。

一辺96px以下になるような画像はフィルター（除外）している。
残った全体の90%を学習、10%をバリデーションに使う。

In [3]:
def dataset_cats_vs_dogs():
    def filter_small(image, _):
        return tf.reduce_all(tf.shape(image)[:2] > tf.constant(96))

    ds_org: tf.data.Dataset
    (ds_org, ), info = tfds.load(name='cats_vs_dogs', with_info=True, as_supervised=True, split=['train'])
    ds_filtered = ds_org.filter(filter_small)

    # filter smaller than 96x96
    num_samples = ds_filtered.reduce(np.int64(0), lambda x, _: x + 1).numpy()
    num_samples_train = int(num_samples * 0.9)
    ds_train = ds_filtered.take(num_samples_train)
    ds_test = ds_filtered.skip(num_samples_train)

    label_names = info.features['label'].names
    return (ds_train, num_samples_train), (ds_test, num_samples - num_samples_train), label_names, 97

`dataset_mydataset`

自分でデータ持ってる用。フィルターなどやっていることは一緒。

学習／バリデーションに分けるのは、シャッフルした後。  
パス情報なら、メモリへの負担は比較的マシやろと乱暴に全体シャッフルしてる。あんまよくないと思う。。

In [4]:
def dataset_mydataset():
    def filter_small(image, _):
        return tf.reduce_all(tf.shape(image)[:2] > tf.constant(96))

    def generator():
        path_iter = itertools.chain(pathlib.Path('train/cat').iterdir(), pathlib.Path('train/dog').iterdir())
        path_list = list(path_iter)
        for path in filter(lambda p: p.suffix in {'.png', '.jpg'}, random.sample(path_list, len(path_list))):
            try:
                image = tf.image.decode_image(tf.io.read_file(str(path)))
            except Exception:
                continue
            yield image, tf.constant(label_str2int[path.parents[0].stem])

    label_str2int = {'cat': 0, 'dog': 1}
    ds_org = tf.data.Dataset.from_generator(generator,
                                            (tf.float32, tf.int32),
                                            (tf.TensorShape([None, None, 3]), tf.TensorShape([])))
    ds_filtered = ds_org.filter(filter_small)

    # filter smaller than 96x96
    num_samples = ds_filtered.reduce(np.int64(0), lambda x, _: x + 1).numpy()
    num_samples_train = int(num_samples * 0.9)
    ds_train = ds_filtered.take(num_samples_train)
    ds_test = ds_filtered.skip(num_samples_train)

    return (ds_train, num_samples_train), (ds_test, num_samples - num_samples_train), ['cat', 'dog'], 97

### 学習

自分のメモリ容量に応じて、`buffer_size`, `batch_size` を調整。

  - buffer_size: メインメモリに関係。
  - batch_size: GPU側メモリに関係。

ラベルは、One-hotベクトルを使うので、`CategoricalCrossentropy`にしている。

epochsはコメントアウトしてある回数を指定する。  
ルールで回数が指定されているので回しきる。EarlyStoppingは使わない。  
※お試しで、10回にしてるだけ。

In [10]:
def train(bit_model: WrappedBiT, dst_dir: pathlib.Path, dataset_factory, cache_dir, buffer_size=None, batch_size=512):
    rule, ds_train, ds_test, label_names = setup_dataset_and_hyperrule(
        dataset_factory, cache_dir, buffer_size=buffer_size, batch_size=batch_size)
    loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    bit_model.compile(optimizer=rule.optimizer, loss=loss_fn, metrics=['accuracy'])

    callbacks = [
        # tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=1),
        tf.keras.callbacks.ModelCheckpoint(filepath=str(dst_dir / 'tmp.ckpt'), verbose=1, save_best_only=True),
        tf.keras.callbacks.TensorBoard(log_dir=str(dst_dir / 'tfboard'), histogram_freq=1, write_images=1),
    ]

    bit_model.fit(
        ds_train,
        batch_size=rule.batch_size,
        steps_per_epoch=10,
        epochs=10, # rule.steps // 10,
        validation_data=ds_test,
        callbacks=callbacks
    )
    bit_model.save(str(dst_dir / 'model'), save_format='tf')
    (dst_dir / 'label_names.json').write_text(json.dumps(label_names, ensure_ascii=False), encoding='utf-8')


def main(version='1'):
    tf.get_logger().setLevel(logging.ERROR)
    for d in tf.config.experimental.list_physical_devices('GPU'):
        tf.config.experimental.set_memory_growth(d, True)

    bit_type = 'm-r50x1'
    model = WrappedBiT(classes=2, bit_type=bit_type)

    # ds_factory = dataset_mydataset
    ds_factory = dataset_cats_vs_dogs

    model_dir = pathlib.Path(f'models/{bit_type}/{version}')
    cache_dir = pathlib.Path('cache/cats_dogs')
    model_dir.mkdir(exist_ok=True, parents=True)
    cache_dir.mkdir(exist_ok=True, parents=True)

    train(model, model_dir, ds_factory, cache_dir, buffer_size=2000, batch_size=64)

In [11]:
main()

Epoch 1/10
10/10 [==============================] - ETA: 0s - loss: 0.3496 - accuracy: 0.8562
Epoch 00001: val_loss improved from inf to 0.04375, saving model to models\m-r50x1\1\tmp.ckpt
10/10 [==============================] - 69s 7s/step - loss: 0.3496 - accuracy: 0.8562 - val_loss: 0.0438 - val_accuracy: 0.9939
Epoch 2/10
10/10 [==============================] - ETA: 0s - loss: 0.2552 - accuracy: 0.9781
Epoch 00002: val_loss improved from 0.04375 to 0.02656, saving model to models\m-r50x1\1\tmp.ckpt
10/10 [==============================] - 60s 6s/step - loss: 0.2552 - accuracy: 0.9781 - val_loss: 0.0266 - val_accuracy: 0.9931
Epoch 3/10
10/10 [==============================] - ETA: 0s - loss: 0.0952 - accuracy: 0.9797
Epoch 00003: val_loss improved from 0.02656 to 0.02466, saving model to models\m-r50x1\1\tmp.ckpt
10/10 [==============================] - 60s 6s/step - loss: 0.0952 - accuracy: 0.9797 - val_loss: 0.0247 - val_accuracy: 0.9935
Epoch 4/10
10/10 [=======================